<a href="https://colab.research.google.com/github/hyunwook-lee/Intern/blob/master/ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 3.4 MB 10.8 MB/s 
     |████████████████████████████████| 6.8 MB 48.4 MB/s 
     |████████████████████████████████| 1.2 MB 52.4 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 895 kB 60.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=25906a7e52f1bab4a8e6058c93ccfbf48538068c698c04180a385ddc081e85de
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy import dot
from numpy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from google.colab import drive
#drive.mount("/content/drive")
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/931 [00:00<?, ?B/s]

In [ ]:
def large_get(train_data_name):
    list_name = list(train_data_name[2])
    for x in range(len(list_name)):
        split_list = list_name[x].split()
        list_name[x] = split_list[0]+" "+split_list[1]
    train_data_name["A"] = list_name
    return train_data_name

def voting(name, category, name_data, category_data):

    # embedding
    embedding_name = model.encode(name)
    embedding_category = model.encode(category)

    #score get
    name_data['score'] = name_data.apply(lambda x: cos_sim(x['embedding'], embedding_name), axis=1)
    category_data['score'] = category_data.apply(lambda x: cos_sim(x['embedding'], embedding_category), axis=1)
    
    #칼럼 추출

    train_data_name = name_data[["A", "score"]]
    train_data_category = category_data[["A", "score"]]

    # sorting
    name_sorting = train_data_name.sort_values(by="score", ascending=False)
    category_sorting = train_data_category.sort_values(by="score", ascending=False)

    # 중복제거
    name_sorting = name_sorting.drop_duplicates(['A'])
    category_sorting = category_sorting.drop_duplicates(['A'])

    # merge
    merge_df = pd.merge(name_sorting,category_sorting, how="outer", on="A")
    merge_df = merge_df.dropna(axis=0)
    merge_df["score"] = merge_df[["score_x", "score_y"]].mean(axis=1)
    merge_df = merge_df.sort_values(by="score", ascending=False)

    # # 나타내기
    # display(name_sorting)
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # display(category_sorting)
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # print("------------------------------------------------------------------")
    # display(merge_df)
    # result = merge_df.iloc[:3]
    # result = result[["A", "score"]]
    # result_list = result.values.tolist()
    # result_list.append([name_sorting.iloc[0]["A"], name_sorting.iloc[0]["score"]])
    # result_list.append([category_sorting.iloc[0]["A"] ,category_sorting.iloc[0]["score"]])
    # print("-----------------------------------------------------------------------")
    # print("입력|| name : ", name, "|| category : ", category)
    # return result_list[0]

    #평가모델시

    return merge_df["A"].iloc[0]
    
    #merge_df.at[0,"A"]
# 27: print(df["현재가"].loc["037730"])
# 28: print(df["현재가"]["037730"])
# 27: print(df["현재가"].iloc[0])
# 28: print(df["현재가"][0])



In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    df_sorting = train_data.sort_values(by="score", ascending=False)
    df_sorting = df_sorting.drop_duplicates(['A'])
    return train_data.loc[train_data['score'].idxmax()]['A']

#jhgan/ko-sroberta-multitask

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
coupang_N2C = pd.read_csv('/content/drive/MyDrive/Colab_data/coupang/Big_thief..csv')
coupang_N2C
test_data = coupang_N2C.rename(columns={"coupang_ItemNm" : "name", "naver_category" : "y", "coupang_category" : "category"})
# test_data = test_data[3500:3600]
test_data

,name,category,y
0,"라운드랩 독도 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
1,"토니모리 원더 세라마이드 모찌 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
2,"닥터지 레드 블레미쉬 클리어 수딩 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
3,"마녀공장 비피다 바이옴 앰플 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
4,"더랩바이블랑두 올리고 히알루론산 5000 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
...,...,...,...
103034,"헬스헬퍼 맥스컷 버닝 1box(병), 1개입 120정",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103035,"자유다방 5일에이드클렌즈 망고맛, 100ml, 5개",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103036,"슈퍼하트 리셋카페 다이어트 커피, 84g, 2박스",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103037,"컷팅 베이스 다이어트 식품, 300g, 1개",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품


In [ ]:
for i in range(1000):
    target_idx = i * 100
    print(target_idx)

    if i == 0:
        df = test_data[target_idx: target_idx + 5]
    
    else: 
        df = df.append(test_data[target_idx: target_idx + 5])
test_data = df
test_data

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

,name,category,y
0,"라운드랩 독도 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
1,"토니모리 원더 세라마이드 모찌 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
2,"닥터지 레드 블레미쉬 클리어 수딩 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
3,"마녀공장 비피다 바이옴 앰플 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
4,"더랩바이블랑두 올리고 히알루론산 5000 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
...,...,...,...
99900,"나우푸드 츄어블 파파야 효소 로젠지, 360개입, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99901,"아이비웰니스 빵순이 효소 석류맛, 30g, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99902,"푸드버킷 트리플 효소, 30개입, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99903,"아이비웰니스 빵순이 효소 키위맛, 30포, 3g",헬스 건강식품 영양제 효소,식품 건강식품


In [ ]:
test_data = test_data[75:]
test_data

,name,category,y
1500,"[공식수입] 클린 웜 코튼 오 드 퍼퓸, 1개, 30ml",뷰티 향수 여성향수,화장품/미용 향수
1501,"랑방 모던 프린세스 오 드 퍼퓸, 30ml",뷰티 향수 여성향수,화장품/미용 향수
1502,"더페이스샵 소울 시크릿 블라썸 향수, 1개, 30ml",뷰티 향수 여성향수,화장품/미용 향수
1503,"메종드펜세 크림 퍼퓸 더 퍼스트 무드, 50ml, 1개",뷰티 향수 여성향수,화장품/미용 향수
1504,"제니퍼로페즈 글로우 바이 제이로 EDT, 50ml, 1개",뷰티 향수 여성향수,화장품/미용 향수
...,...,...,...
99900,"나우푸드 츄어블 파파야 효소 로젠지, 360개입, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99901,"아이비웰니스 빵순이 효소 석류맛, 30g, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99902,"푸드버킷 트리플 효소, 30개입, 1개",헬스 건강식품 영양제 효소,식품 건강식품
99903,"아이비웰니스 빵순이 효소 키위맛, 30포, 3g",헬스 건강식품 영양제 효소,식품 건강식품


In [ ]:
# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
# pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None)

In [ ]:

train_data_name = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/N2C(naver100000).pkl')
C2C = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/C2C (1).pkl')
train_data_name = large_get(train_data_name)
N2C = train_data_name[["Q", "A", "embedding"]]


def voting_test_model(test_data, predict_fuction):

  score = 0
  test_data = test_data.copy()
  for i in range(len(test_data.y)):
    if( i %10 == 0):
        print(f"{i}번째")
    name = test_data.copy().name.iloc[i]
    category = test_data.copy().category.iloc[i]
    y = test_data.copy().y.iloc[i]
    predict = predict_fuction(name, category, N2C, C2C)

    if y == predict:
      score += 1
    
    else: 
      print('='*100)
      print('오답')
      print(f'상품명: {name}')
      print(f'예측값: {predict}')
      print(f'실제값: {y}')

  print(f'해당 모델 성능: {score / len(test_data.y)}')

  return score
voting_test_model(test_data, voting)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
오답
상품명:     이불압축팩 특대형 5매 랜덤발송, 1매입, 10개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
오답
상품명:     조은리빙 이불압축팩 소형 벨브형 랜덤 발송, 1개입, 10개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
오답
상품명:     조은리빙 이불압축팩 대형 벨브형 랜덤 발송, 5개입, 1개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
오답
상품명:     마켓플랜 메쉬 가방정리 보관함 4단 아이보리, 1개입, 1개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
오답
상품명:     리벤스 매직 의류 압축팩 대형 밸브형, 8매, 1개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
1880번째
오답
상품명:     도우홈라이프 자석 고리 후크, 10개
예측값: 생활/건강 수납/정리용품
실제값: 가구/인테리어 수납가구
오답
상품명:     케이트가든 주방도구 소품 자석걸이 벽고리 3p 세트, 1세트
예측값: 가구/인테리어 주방가구
실제값: 가구/인테리어 수납가구
오답
상품명:     에이트룸 프리미엄 10mm 라셀 겨울 극세사 이불, 그레이
예측값: 가구/인테리어 침구세트
실제값: 가구/인테리어 침구단품
오답
상품명:     마틸라 디저트 차렵이불 단품, 코코옐로우
예측값: 가구/인테리어 침구세트
실제값: 가구/인테리어 침구단품
1890번째
오답
상품명:     센스맘 매트리스 시그니처9 10cm, 어반그레이
예측값: 가구/인테리어 침실가구
실제값: 가구/인테리어 침구단품
오답
상품명:     구르미 9존 프레쉬폼 접이식 매트리스, 그레이
예측값: 가구/인테리어 침구세트
실제값: 가구/인테리어 침구단품
1900번째
오답
상품명:     라뽐므 노르빅 호텔식 베개커버 2p, 차콜그레이
예측값: 가구/인테리어 침구세트
실제값: 가구/인테리어 침구단품
오답
상품명:     

3421

In [ ]:
kan_naver_cat = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/category to category.csv', encoding="cp949")
kan_naver_cat.columns = [0,"Q","A"]
kan_naver_cat

,0,Q,A
0,0.0,가공식품 조미료 종합조미료 천연/발효조미료,식품 가공식품
1,1.0,가공식품 조미료 종합조미료 식초,식품 가공식품
2,2.0,가공식품 조미료 종합조미료 천일염,식품 가공식품
3,3.0,가공식품 조미료 종합조미료 가공염,식품 가공식품
4,4.0,가공식품 조미료 종합조미료 설탕,식품 가공식품
...,...,...,...
2872,2869.0,전문스포츠/레저 무술 무술용품,스포츠/레저 기타스포츠용품
2873,NaN,가구/인테리어 서재/사무용가구 책장,가구/인테리어 서재/사무용가구
2874,NaN,가구/인테리어 서재/사무용가구 책상,가구/인테리어 서재/사무용가구
2875,NaN,가구/인테리어 서재/사무용가구 의자,가구/인테리어 서재/사무용가구


In [ ]:
# 칸 카테고리 to 네이버 중분류 카테고리
kan_naver_cat
kan_naver_cat['embedding'] = kan_naver_cat.apply(lambda row: model.encode(row.Q), axis = 1)
train_data = kan_naver_cat

In [ ]:
# 저장 로직
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/C2C.pkl')

In [ ]:
#그냥 name to category
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

#10만개 name to category


In [ ]:
# train_data_name = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/naver_name_to_category.pkl')


,0,1,2,Q,3,A,embedding
0,0,0,스포츠/레저 마라톤용품,나이키 스파크 라이트웨이트,나이키공식홈페이지,스포츠/레저 마라톤용품,"[-0.2385571, 0.0755591, 0.6072318, -0.29075518..."
1,1,1,스포츠/레저 마라톤용품,런닝 육상복 민소매 유니폼 마라톤복 세트,더디씨몰,스포츠/레저 마라톤용품,"[-0.0062362906, 0.57027984, 0.02965233, -0.671..."
2,2,2,스포츠/레저 마라톤용품,플립벨트 FLIPBELT 런닝벨트 런닝밴드 러닝 스마트폰허리가방 마라톤힙색,올유니드스토어,스포츠/레저 마라톤용품,"[0.11287337, 0.2088058, -0.05426197, -0.386079..."
3,3,3,스포츠/레저 마라톤용품,프리가든 러닝벨트 런닝 휴대폰 플립벨트 스마트폰 허리 밴드 벨크로 러닝백,프리가든,스포츠/레저 마라톤용품,"[0.20414239, 0.16623603, -0.14755379, -0.11114..."
4,4,4,스포츠/레저 마라톤용품,무지 벨크로 러닝벨트 플립벨트 밴드 스마트폰 허리 러닝용품,찰리 스토어,스포츠/레저 마라톤용품,"[0.55654615, -0.03387138, -0.32060072, 0.24906..."
...,...,...,...,...,...,...,...
103173,103885,103885,출산/육아 유아동잡화 기타유아동잡화,몽베베 올리브빕,윤스룸,출산/육아 유아동잡화,"[0.2544918, -0.42845765, -0.0893433, 0.5567667..."
103174,103886,103886,출산/육아 유아동잡화 기타유아동잡화,빛나는구름 한복 소품 복주머니,꼬마누나,출산/육아 유아동잡화,"[0.78941107, -0.06786195, 0.40254658, 0.281634..."
103175,103887,103887,출산/육아 유아동잡화 기타유아동잡화,미소 패피선글라스,아리키즈,출산/육아 유아동잡화,"[0.22618075, 0.24640131, -0.04846, 0.62539375,..."
103176,103888,103888,출산/육아 유아동잡화 기타유아동잡화,장갑M 샌드박스 폴라오지 잠뜰,NaN,출산/육아 유아동잡화,"[-0.21964131, -0.14713143, 0.61405224, 0.66292..."


In [ ]:
return_similar_answer('홈플래닛 고속충전 케이블 5핀 5색 1.5m 5개')

NameError: ignored

# 3000개 Kan category to naver middle category

In [ ]:
train_data_category = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/C2C.pkl')
train_data = train_data_category


,coupang_ItemNm,coupang_category,naver_category
0,"라운드랩 독도 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
1,"토니모리 원더 세라마이드 모찌 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
2,"닥터지 레드 블레미쉬 클리어 수딩 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
3,"마녀공장 비피다 바이옴 앰플 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
4,"더랩바이블랑두 올리고 히알루론산 5000 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
...,...,...,...
103034,"헬스헬퍼 맥스컷 버닝 1box(병), 1개입 120정",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103035,"자유다방 5일에이드클렌즈 망고맛, 100ml, 5개",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103036,"슈퍼하트 리셋카페 다이어트 커피, 84g, 2박스",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품
103037,"컷팅 베이스 다이어트 식품, 300g, 1개",헬스 건강식품 다이어트 이너뷰티 기타다이어트식품,식품 다이어트식품


In [ ]:
list123 = ['가구/인테리어 서재/사무용가구', '가구/인테리어 솜류', '가구/인테리어 수예', '가구/인테리어 시계', '가구/인테리어 아웃도어가구', '가구/인테리어 장갑', '가구/인테리어 카페트/러그', '가구/인테리어 홈데코', '디지털/가전 광학기기/용품', '생활/건강 관상어용품', '생활/건강 기저귀', '생활/건강 김치', '생활/건강 당뇨관리용품', '생활/건강 모자', '생활/건강 물리치료/저주파용품', '생활/건강 발건강용품', '생활/건강 벨트', '생활/건강 블루레이', '생활/건강 솜류', '생활/건강 수납/정리용품', '생활/건강 실버용품', '생활/건강 안전용품', '생활/건강 인형', '생활/건강 장갑', '생활/건강 재활운동용품', '생활/건강 종교', '생활/건강 지갑', '생활/건강 화방용품', '생활/건강 휴대폰', '스포츠/레저 검도', '스포츠/레저 권투', '스포츠/레저 댄스', '스포츠/레저 모자', '스포츠/레저 무술용품', '스포츠/레저 벨트', '스포츠/레저 보호용품', '스포츠/레저 수련용품', '스포츠/레저 스킨스쿠버', '스포츠/레저 양말', '스포츠/레저 장갑', '출산/육아 모자', '출산/육아 목욕용품', '출산/육아 벨트', '출산/육아 소독/살균용품', '출산/육아 수영복/용품', '출산/육아 신생아의류', '출산/육아 양말', '출산/육아 유아동잡화', '출산/육아 임부복', '출산/육아 장갑', '출산/육아 지갑', '출산/육아 출산/돌기념품', '패션잡화 순금', '패션잡화 주얼리']

for x in range(len(list123)):
    print(list123[x])
    print(return_similar_answer(list123[x]))
    print("-------------------------------------------")

가구/인테리어 서재/사무용가구
가구/인테리어 서재/사무용가구
-------------------------------------------
가구/인테리어 솜류
가구/인테리어 솜류
-------------------------------------------
가구/인테리어 수예
가구/인테리어 수예
-------------------------------------------
가구/인테리어 시계
가구/인테리어 인테리어소품
-------------------------------------------
가구/인테리어 아웃도어가구
가구/인테리어 아웃도어가구
-------------------------------------------
가구/인테리어 장갑
가구/인테리어 DIY자재/용품
-------------------------------------------
가구/인테리어 카페트/러그
가구/인테리어 DIY자재/용품
-------------------------------------------
가구/인테리어 홈데코
가구/인테리어 수납가구
-------------------------------------------
디지털/가전 광학기기/용품
디지털/가전 광학기기/용품
-------------------------------------------
생활/건강 관상어용품
생활/건강 화방용품
-------------------------------------------
생활/건강 기저귀
출산/육아 외출용품
-------------------------------------------
생활/건강 김치
식품 김치
-------------------------------------------
생활/건강 당뇨관리용품
생활/건강 의료용품
-------------------------------------------
생활/건강 모자
생활/건강 안전용품
-------------------------------------------
생활/건강 물리치료/저주파용품
생활/건강 안마용품
----

In [ ]:
return_similar_answer('여성패션 의류 원피스')

NameError: ignored

In [ ]:
data_name = 
data_category = 

print(data_name)
print(data_category)
count = 0
for x in range(len(data_category)):
    for y in range(len(data_name)):
        if (data_name[y] in data_category[x]):
            count += 1
            data_name[y] = data_category[x]
            print(data_category[x])
print(data_name)
print(count)

NameError: ignored

In [ ]:
# test_data: (type: DataFrame, des: column{x, y})
# predict_fuction: (type: fuction, des: input: x(query), output: predict(matching with y) )

def test_model(test_data, predict_fuction):

  score = 0

  for i in range(len(test_data.x)):

    x = test_data.x.iloc[i]
    y = test_data.y.iloc[i]
    predict = predict_fuction(x)

    if y == predict:
      score += 1
    
    else: 
      print('='*100)
      print('오답')
      print(f'상품명: {x}')
      print(f'예측값: {predict}')
      print(f'실제값: {y}')

  print(f'해당 모델 성능: {score / len(test_data.x)}')

  return score

,name,category,y
0,"라운드랩 독도 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
1,"토니모리 원더 세라마이드 모찌 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
2,"닥터지 레드 블레미쉬 클리어 수딩 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
3,"마녀공장 비피다 바이옴 앰플 토너, 500ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
4,"더랩바이블랑두 올리고 히알루론산 5000 토너, 300ml, 1개",뷰티 스킨케어 스킨,화장품/미용 스킨케어
...,...,...,...
995,"오제끄 모스트 모이스트 산소 마스크 클렌저, 120ml, 2개",뷰티 클렌징 필링 클렌징 로션 크림,화장품/미용 클렌징
996,"오휘 미라클 모이스처 클렌징 크림, 200ml, 1개",뷰티 클렌징 필링 클렌징 로션 크림,화장품/미용 클렌징
997,"미샤 수퍼 아쿠아 울트라 히알론 클렌징 크림, 200ml, 1개",뷰티 클렌징 필링 클렌징 로션 크림,화장품/미용 클렌징
998,"헤이미쉬 올클린밤, 120ml, 4개",뷰티 클렌징 필링 클렌징 로션 크림,화장품/미용 클렌징


In [ ]:
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/coupang_category.csv', encoding="cp949")
test_data

,0,1,Unnamed: 2
0,뷰티 스킨케어 스킨,화장품/미용 스킨케어,NaN
1,뷰티 스킨케어 로션,화장품/미용 스킨케어,NaN
2,뷰티 스킨케어 에센스 세럼 앰플,화장품/미용 스킨케어,NaN
3,뷰티 스킨케어 미스트,화장품/미용 스킨케어,NaN
4,뷰티 스킨케어 오일,화장품/미용 스킨케어,NaN
...,...,...,...
1708,헬스 건강식품 다이어트 이너뷰티 다이어트 쉐이크,식품 다이어트식품,NaN
1709,헬스 건강식품 다이어트 이너뷰티 과일 야채음료,식품 다이어트식품,NaN
1710,헬스 건강식품 다이어트 이너뷰티 히알루론산 콜라겐,식품 다이어트식품,NaN
1711,헬스 건강식품 다이어트 이너뷰티 석류,식품 다이어트식품,NaN


In [ ]:
# test_data = test_data.rename(columns={"0" : "x", "1" : "y"})
# train_data_category = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/C2C.pkl')
# train_data = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/N2C(naver100000).pkl')
test_model(test_data, return_similar_answer)

오답
상품명:     닥터지 레드 블레미쉬 클리어 수딩 토너, 300ml, 1개
예측값: 화장품/미용 클렌징
실제값: 화장품/미용 스킨케어
오답
상품명:     니들리 데일리 토너패드 60매입 + 앰플 2ml x 5p + 워터젤크림 1.5ml x 5p, 1세트
예측값: 화장품/미용 뷰티소품
실제값: 화장품/미용 스킨케어
오답
상품명:     토리든 다이브인 저분자 히알루론산 토너, 300ml, 1개
예측값: 화장품/미용 뷰티소품
실제값: 화장품/미용 스킨케어
오답
상품명:     더모테라피 시카산 리페어레이션 에센스 토너, 1L, 1개
예측값: 화장품/미용 뷰티소품
실제값: 화장품/미용 스킨케어
오답
상품명:     애터미 더페임 토너, 150ml, 1개
예측값: 화장품/미용 남성화장품
실제값: 화장품/미용 스킨케어
오답
상품명:     일리윤 세라마이드 더마 속보습 스킨, 180ml, 1개
예측값: 화장품/미용 선케어
실제값: 화장품/미용 스킨케어
오답
상품명:     록키스 워터밤 빙하수 50% 약산성 모공 토너, 500ml, 1개
예측값: 화장품/미용 마스크/팩
실제값: 화장품/미용 스킨케어
오답
상품명:     이니스프리 청보리 필링 토너, 250ml, 1개
예측값: 화장품/미용 클렌징
실제값: 화장품/미용 스킨케어
오답
상품명:     모공도둑 콜라겐 토너, 200ml, 1개
예측값: 화장품/미용 클렌징
실제값: 화장품/미용 스킨케어
오답
상품명:     라네즈 크림 스킨 150ml + 아모레퍼시픽 기초케어 4종, 1세트
예측값: 화장품/미용 베이스메이크업
실제값: 화장품/미용 스킨케어
오답
상품명:     토니모리 어성초 시카 퀵 카밍 97 토너, 1개, 300ml
예측값: 화장품/미용 클렌징
실제값: 화장품/미용 스킨케어
오답
상품명:     코스알엑스 아하 바하 클래리파잉 트리트먼트 토너, 150ml, 1개
예측값: 화장품/미용 남성화장품
실제값: 화장품/미용 스킨케어
오답
상품명:     키로 히아루론산 모이스춰 스킨, 1000g, 1개
예측값

KeyboardInterrupt: ignored

In [ ]:
train_data = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/N2C(naver100000).pkl')
train_data

cat_all = list(train_data[2])
cat_middle = list(train_data["A"])

for x in range(len(cat_middle)):
    cat_middle[x] = cat_all[x].split()[0]+" "+cat_middle[x]
cat_middle = pd.DataFrame(cat_middle)
train_data["middle_classification"] = cat_middle

train_data = train_data.rename(columns={"A" : "classify", "middle_classification" : "A"})

,0,1,2,Q,3,classify,embedding,A
0,0,0,스포츠/레저 마라톤용품,나이키 스파크 라이트웨이트,나이키공식홈페이지,마라톤용품,"[-0.2385571, 0.0755591, 0.6072318, -0.29075518...",스포츠/레저 마라톤용품
1,1,1,스포츠/레저 마라톤용품,런닝 육상복 민소매 유니폼 마라톤복 세트,더디씨몰,마라톤용품,"[-0.0062362906, 0.57027984, 0.02965233, -0.671...",스포츠/레저 마라톤용품
2,2,2,스포츠/레저 마라톤용품,플립벨트 FLIPBELT 런닝벨트 런닝밴드 러닝 스마트폰허리가방 마라톤힙색,올유니드스토어,마라톤용품,"[0.11287337, 0.2088058, -0.05426197, -0.386079...",스포츠/레저 마라톤용품
3,3,3,스포츠/레저 마라톤용품,프리가든 러닝벨트 런닝 휴대폰 플립벨트 스마트폰 허리 밴드 벨크로 러닝백,프리가든,마라톤용품,"[0.20414239, 0.16623603, -0.14755379, -0.11114...",스포츠/레저 마라톤용품
4,4,4,스포츠/레저 마라톤용품,무지 벨크로 러닝벨트 플립벨트 밴드 스마트폰 허리 러닝용품,찰리 스토어,마라톤용품,"[0.55654615, -0.03387138, -0.32060072, 0.24906...",스포츠/레저 마라톤용품
...,...,...,...,...,...,...,...,...
995,995,995,스포츠/레저 기타스포츠용품,3단뜀틀 가벼운것 F24A 뜀틀 신체활동능력향상,쇼핑몰별 최저가,기타스포츠용품,"[-0.6804353, 0.014544737, 0.20192797, -0.06243...",스포츠/레저 기타스포츠용품
996,996,996,스포츠/레저 기타스포츠용품,미니 하트 눈뭉치제 조기 스노우볼 메이커 눈사람 만들기 눈제 크리스마스 선물 오리 집게,쇼핑몰별 최저가,기타스포츠용품,"[-0.15816891, 0.4099266, 0.52104664, -0.387778...",스포츠/레저 기타스포츠용품
997,997,997,스포츠/레저 기타스포츠용품,스타일리쉬 승마 보호 조끼 어린이 성인 보호장비,NaN,기타스포츠용품,"[-0.16006242, 0.41424465, 0.12419352, 1.041158...",스포츠/레저 기타스포츠용품
998,998,998,스포츠/레저 기타스포츠용품,스타 릴레이 바톤 세트 육상 이어달리기 바톤 폼 6p,쇼핑몰별 최저가,기타스포츠용품,"[-0.561603, 0.7376249, -0.2883688, 0.12811267,...",스포츠/레저 기타스포츠용품
